In [25]:
import graphviz
import json
import sqlite3
import sys, os, argparse
from IPython.display import display


In [32]:
def _run_sql_query(db, query):
    conn = sqlite3.connect(db)
    cur = conn.cursor()
    cur.execute(query)
    result_data = json.dumps(cur.fetchall())
    result_json = json.loads(result_data)
    return result_json

def _gen_records(graph, nodes, edge_tuples):
    graph.attr('node', shape='record', fontname='Courier Prime', size='6,6')
    graph.node_attr['fontname'] = "Courier Prime"
    graph.node_attr['color'] = "lightgreen"
    graph.node_attr['fillcolor'] = "lightblue"
    graph.node_attr['style'] = 'filled'
    graph.edge_attr['fontname'] = "Courier Prime"
    graph.graph_attr['rankdir'] = "RL"
    graph.node_attr['fontsize'] = "10"
        #subg.node_attr['shape'] = "ellipse"
        
        #for i in range(len(vm_nodes)):
    for node in nodes:
        graph.node(node, nodes[node])
        
    edges = []
    for edge_tuple in edge_tuples:
        graph.edge(edge_tuple[0], edge_tuple[1], label=edge_tuple[2])
    
def _gen_node(node_id, node_txt):
    nodes = {}
    nodes[node_id] = node_txt
    return nodes


In [33]:
def _total_num_of_caps(db):
    count_caps_q = "SELECT COUNT(*) FROM cap_info"
    return _run_sql_query(db, count_caps_q)


In [46]:
def _binary_graph(db, graph):
    get_bin_paths_q = "SELECT DISTINCT mmap_path FROM vm"
    path_list_json = _run_sql_query(db, get_bin_paths_q)
    
    get_caps_q = "SELECT * FROM cap_info"
    caps = _run_sql_query(db, get_caps_q)
                
    nodes = {}
    edges = []
    
    for path_list in path_list_json:
        nodes.update(_gen_node(path_list[0], path_list[0]))
        
        lib_start_q = "SELECT start_addr FROM vm WHERE mmap_path LIKE '%" + str(path_list[0]) + "%'"
        lib_end_q = "SELECT end_addr FROM vm WHERE mmap_path LIKE '%" + str(path_list[0]) + "%'"
        lib_start_addrs = _run_sql_query(db, lib_start_q)
        lib_end_addrs = _run_sql_query(db, lib_end_q)
        
        for cap in caps:
            cap_loc_addr = cap[0] 
            cap_path = cap[1]
            cap_addr = cap[2]
            cap_perms = cap[3]
            cap_base = cap[4]
            cap_top = cap[5]

            for lib_addr_index in range(len(lib_start_addrs)):
                lib_start_addr = lib_start_addrs[lib_addr_index][0]
                lib_end_addr = lib_end_addrs[lib_addr_index][0]
                
                if cap_addr >= lib_start_addr and cap_addr <= lib_end_addr and cap_path != path_list[0]:
                    if (cap_path, path_list[0], cap_perms) not in edges:            
                        edges.append((cap_path, path_list[0], cap_perms))
    
    _gen_records(graph, nodes, edges)


In [47]:
def _show_caps_to_bin(db, path, graph):
    get_caps_q = "SELECT * FROM cap_info"
    caps = _run_sql_query(db, get_caps_q)
    
    lib_start_q = "SELECT start_addr FROM vm WHERE mmap_path LIKE '%" + str(path) + "%'"
    lib_end_q = "SELECT end_addr FROM vm WHERE mmap_path LIKE '%" + str(path) + "%'"
    lib_start_addrs = _run_sql_query(db, lib_start_q)
    lib_end_addrs = _run_sql_query(db, lib_end_q)
    
    # node showing the loaded library, with cap records pointing into it
    # We have all the information here at this point, just need to graph it
    
    # First we create the library node
    nodes = {}
    edges = []
    
    nodes.update(_gen_node(path, path))
    
    for cap in caps:
        cap_loc_addr = cap[0] 
        cap_path = cap[1]
        cap_addr = cap[2]
        cap_perms = cap[3]
        cap_base = cap[4]
        cap_top = cap[5]
        
        if path in cap_path:
            node_label = cap_addr
            cap_node = _gen_node(cap_addr, cap_addr +"|"+cap_perms+"|"+cap_base+"-"+cap_top)
            nodes.update(cap_node)

    _gen_records(graph, nodes, edges)


In [48]:
def _show_caps_between_two_libs(db, lib1, lib2, graph):
    lib1_caps_q = "SELECT * FROM cap_info WHERE cap_loc_path LIKE '%" + str(lib1) + "%'"
    lib1_caps = _run_sql_query(db, lib1_caps_q)
    
    lib2_caps_q = "SELECT * FROM cap_info WHERE cap_loc_path LIKE '%" + str(lib2) + "%'"
    lib2_caps = _run_sql_query(db, lib2_caps_q)
    
    lib1_start_q = "SELECT start_addr FROM vm WHERE mmap_path LIKE '%" + str(lib1) + "%'"
    lib1_end_q = "SELECT end_addr FROM vm WHERE mmap_path LIKE '%" + str(lib1) + "%'"
    lib1_start_addrs = _run_sql_query(db, lib1_start_q)
    lib1_end_addrs = _run_sql_query(db, lib1_end_q)
    
    lib2_start_q = "SELECT start_addr FROM vm WHERE mmap_path LIKE '%" + str(lib2) + "%'"
    lib2_end_q = "SELECT end_addr FROM vm WHERE mmap_path LIKE '%" + str(lib2) + "%'"
    lib2_start_addrs = _run_sql_query(db, lib2_start_q)
    lib2_end_addrs = _run_sql_query(db, lib2_end_q)
    
    # node showing the loaded libraries, with cap records pointing into it
    # We have all the information here at this point, just need to graph it
    
    # First we create the two library nodes
    nodes = {}
    edges = []
    
    nodes.update(_gen_node(lib1, lib1))
    nodes.update(_gen_node(lib2, lib2))
    
    for cap1 in lib1_caps:
        cap_loc_addr = cap1[0] 
        cap_path = cap1[1]
        cap_addr = cap1[2]
        cap_perms = cap1[3]
        cap_base = cap1[4]
        cap_top = cap1[5]
        
        for lib2_addr_index in range(len(lib2_start_addrs)):
            lib2_start_addr = lib2_start_addrs[lib2_addr_index][0]
            lib2_end_addr = lib2_end_addrs[lib2_addr_index][0]
                
            if cap_addr >= lib2_start_addr and cap_addr <= lib2_end_addr:
                if (lib1, lib2, cap_perms) not in edges:            
                    edges.append((lib1, lib2, cap_perms))
                        
    for cap2 in lib2_caps:
        cap_loc_addr = cap2[0] 
        cap_path = cap2[1]
        cap_addr = cap2[2]
        cap_perms = cap2[3]
        cap_base = cap2[4]
        cap_top = cap2[5]
        
        for lib1_addr_index in range(len(lib1_start_addrs)):
            lib1_start_addr = lib1_start_addrs[lib1_addr_index][0]
            lib1_end_addr = lib1_end_addrs[lib1_addr_index][0]
                
            if cap_addr >= lib1_start_addr and cap_addr <= lib1_end_addr:
                if (lib2, lib1, cap_perms) not in edges:            
                        edges.append((lib2, lib1, cap_perms))

    _gen_records(graph, nodes, edges)

In [51]:
CONFIG_FILE = 'chericat_cli.config'

def main(argv):
    if (os.path.isfile(CONFIG_FILE)):
        with open(CONFIG_FILE) as f:
            sys.argv = f.read().split(',')
    else:
#         sys.argv = ['chericat_cli', '-d', '/home/psjm3/chericat/nginx_worker.db', '-g', '-l', 'nginx', '-q', 'SELECT COUNT(*) FROM elf_sym WHERE source_path LIKE "%nginx%"']
#         sys.argv = ['chericat_cli', '-d', '/home/psjm3/chericat/trial.db', '-l', 'print-pointer']
        sys.argv = ['chericat_cli', '-d', '/home/psjm3/chericat/nginx_worker.db', '-c', 'libssl', 'libcrypto']
#         sys.argv = ['chericat_cli', '-d', '/home/psjm3/chericat/nginx_worker.db', '-g']


    parser = argparse.ArgumentParser(prog='chericat_cli')
    parser.add_argument(
        '-d', 
        help='The database to use for the queries', 
        required=True,
    )
    parser.add_argument(
        '-g', 
        help='Loaded binaries graph', 
        action='store_true',
    )

    parser.add_argument(
        '-l', 
        help='Show capabilities pointed to a loaded library <path>',
        nargs=1,
    )
    
    parser.add_argument(
        '-q', 
        help='Executes the SQL query on the provided db',
        nargs=1,
    )
    
    parser.add_argument(
        '-c',
        help="Show capabilities between two loaded libraries <libname 1> <libname 2>",
        nargs=2,
    )

    args = parser.parse_args()
    
    if args.d:
        db = args.d

    if args.g:
        digraph = graphviz.Digraph('G', filename='graph_overview.gv')
        _binary_graph(db, digraph)
        digraph.render(directory='graph-output', view=True)  
        
    if args.l:
        digraph = graphviz.Digraph('G', filename='graph_for_'+args.l[0]+'.gv')
        _show_caps_to_bin(db, args.l[0], digraph)
        digraph.render(directory='graph-output', view=True)  
        
    if args.q:
        print(_run_sql_query(db, args.q[0]))
        
    if args.c:
        digraph = graphviz.Digraph('G', filename=args.c[0]+'_vs_'+args.c[1]+'.gv')
        _show_caps_between_two_libs(db, args.c[0], args.c[1], digraph)
        digraph.render(directory='graph-output', view=True)  

    return None

if __name__ == '__main__':
    main(sys.argv)


FileNotFoundError: [Errno 2] No such file or directory: 'xdg-open'